In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import time
import copy
import numpy as np
import pandas as pd
import chainer
import chainer.functions as F
import chainer.links as L
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
import csv
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

veri= pd.read_csv("./Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv", delimiter=',', skiprows=0, low_memory=False)
veri.head()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [2]:
veri[' Label'].replace(['BENIGN', 'DDoS'], [0, 1], inplace=True)
moddf = veri.dropna()
if ' Flow Packets/s' in moddf.columns:
    del moddf[' Flow Packets/s']

C:\Users\HP-PC\AppData\Local\Temp\ipykernel_20640\1267777773.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\HP-PC\AppData\Local\Temp\ipykernel_20640\1267777773.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [3]:
moddf.shape

(225741, 78)

In [5]:
features = [col for col in moddf.columns if col != ' Label']
target = ' Label'

In [6]:
X = moddf[features].copy()
y = moddf[target]

In [7]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Optionally, replace NaNs with the mean or median of the column
X.fillna(X.mean(), inplace=True)

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

In [10]:
explained_variance = np.cumsum(pca.explained_variance_ratio_)
n_components = np.argmax(explained_variance >= 0.95) + 1  # Number of components to retain 95% of variance

print(f"Number of components to retain 95% of variance: {n_components}")

Number of components to retain 95% of variance: 21


In [11]:
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X_scaled)

In [23]:
print(X_reduced)

[[-2.20585622 -2.04994977  1.39056357 ... -3.34058955 -2.3387496
   0.07269517]
 [-1.9502359  -2.34685871  1.42217181 ... -0.34457804  0.32625124
  -0.02368959]
 [-1.95937126 -2.35747834  1.43162511 ... -0.27722942  0.3083905
  -0.01443655]
 ...
 [-1.9285787  -2.3892009   1.44395849 ... -0.3499681   0.2811905
  -0.02620324]
 [-2.08523673 -1.88848932  1.27847208 ... -0.56707065 -0.13164066
   0.00950919]
 [-2.04815973 -2.24166183  1.67325458 ... -2.66894542  0.89064803
  -1.07439406]]


In [25]:
xtrain, xtest, ytrain, ytest = tts(X_reduced, y, test_size=0.2, random_state=42)

In [26]:
print(f"NaN values in training data: {np.isnan(xtrain).sum()}")
print(f"NaN values in testing data: {np.isnan(xtest).sum()}")
print(f"NaN values in training labels: {np.isnan(ytrain).sum()}")
print(f"NaN values in testing labels: {np.isnan(ytest).sum()}")

NaN values in training data: 0
NaN values in testing data: 0
NaN values in training labels: 0
NaN values in testing labels: 0


In [27]:
print(np.array(ytest))

[0 0 1 ... 0 1 0]


In [28]:

import tensorflow as tf
from keras.layers import Dense, BatchNormalization, Dropout, LSTM, Bidirectional
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping

In [29]:
#Early stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [32]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=10, return_sequences=True, input_shape=(x_train.shape[1], 1))))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=10, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=10, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=10))
regressor.add(Dropout(0.2))
regressor.add(Dense(units=1, activation='sigmoid'))

In [33]:
regressor.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])

In [34]:
history = regressor.fit(x_train, ytrain, epochs=10, batch_size=128)

Epoch 1/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 63s 35ms/step - acc: 0.8822 - loss: 0.2614
Epoch 2/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - acc: 0.9970 - loss: 0.0146
Epoch 3/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 56s 40ms/step - acc: 0.9975 - loss: 0.0121
Epoch 4/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 52s 37ms/step - acc: 0.9977 - loss: 0.0097
Epoch 5/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 50s 36ms/step - acc: 0.9981 - loss: 0.0081
Epoch 6/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 50s 35ms/step - acc: 0.9981 - loss: 0.0082
Epoch 7/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 50s 36ms/step - acc: 0.9986 - loss: 0.0066
Epoch 8/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 50s 36ms/step - acc: 0.9984 - loss: 0.0069
Epoch 9/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 50s 36ms/step - acc: 0.9987 - loss: 0.0058
Epoch 10/10
1411/1411 ━━━━━━━━━━━━━━━━━━━━ 52s 37ms/step - acc: 0.9986 - loss: 0.0066


In [35]:
trainloss = regressor.evaluate(x_train, ytrain, verbose=0)
testloss = regressor.evaluate(x_test, ytest, verbose=0)

In [22]:
predict = regressor.predict(x_test)

1411/1411 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step


In [23]:
predict_binary = (predict > 0.5).astype(int)

10

In [24]:
predDf = pd.DataFrame(np.array(ytest), columns=["Real_Values"])
predDf2 = pd.Series(predict.reshape(-1,), name='Predicted_Values')

array([[4.5595895e-03],
       [1.2939434e-04],
       [9.9998152e-01],
       ...,
       [2.3563582e-05],
       [7.0220507e-05],
       [9.9997634e-01]], dtype=float32)

In [25]:
pred_results = pd.concat([predDf, predDf2], axis=1)

In [26]:
predDf

,Real predictions:
0,0
1,0
2,1
3,0
4,0
...,...
45144,0
45145,0
45146,0
45147,0


In [27]:
print(pred_results.head())

0        0.004560
1        0.000129
2        0.999982
3        0.000025
4        0.000023
           ...   
45144    0.000024
45145    0.000051
45146    0.000024
45147    0.000070
45148    0.999976
Length: 45149, dtype: float32

In [28]:
print("Accuracy: ", accuracy_score(ytest, predict_binary))
print("Precision: ", precision_score(ytest, predict_binary))
print("Recall: ", recall_score(ytest, predict_binary))
print("F1 Score: ", f1_score(ytest, predict_binary))

,Real predictions:,0
0,0,0.004560
1,0,0.000129
2,1,0.999982
3,0,0.000025
4,0,0.000023
...,...,...
45144,0,0.000024
45145,0,0.000051
45146,0,0.000024
45147,0,0.000070


In [29]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=10, return_sequences=True, input_shape = (x_train.shape[1],1) ) )) #çıktı uzayının boyutu
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 10))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1,activation='sigmoid'))
regressor.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])

In [30]:
history=regressor.fit(x_train, ytrain, epochs=3,batch_size=64 )

,Real_Values,Predicted_Values
0,0,0.004560
1,0,0.000129
2,1,0.999982
3,0,0.000025
4,0,0.000023
...,...,...
45144,0,0.000024
45145,0,0.000051
45146,0,0.000024
45147,0,0.000070


In [31]:
plt.plot(history.history['acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()